In [ ]:
# default_exp core.dashboards

# core.dashboards
> Supplies dashboards and dashboard components. Dashboards are defined as classes, to show the dashboard use the .show() function on an dashboard instance.

In [ ]:
#export
from typing import List, Union, Optional
from abc import ABC, abstractmethod
from math import floor, ceil
import os

import numpy as np
import pandas as pd
import panel as pn
import panel.widgets as pnw

from icevision_dashboards.plotting import *
from icevision_dashboards.core.data import *

In [ ]:
#hide
pn.extension()

In [ ]:
#export
class Dashboard(ABC):
    def __init__(self, width: int = 500, height: int = 500):
        self.width = width
        self.height = height
        self.build_gui()
        
    @abstractmethod
    def show(self):
        pass
    
    @abstractmethod
    def build_gui(self):
        pass

Abstaract base class for dashboards. Inherited classes are required to implement a `build_gui` function, that builds the gui to be shown as well as setting up interactions and a `show` function that returns the gui. The `build_gui` function is called at the end of the `__init__` function. Most dashboards require different descriptors from the dataset (data, stats usw.). For simple handling of cases where the name is different (a vision dataset might have stats for the images, the annotations and the annotation classes) most dashboards have class variables for the different descriptors, wich can be changed to use a new descriptor.

For complexer dashboards, that use other dashboards as components the same approche is used, where class varibales ar used to choose a implementation of a dashboard. With this, in some cases only certain parts need to be replaced and not everything.

In [ ]:
#hide
class TestDataset(GenericDataset):
    def __init__(self, data=None, name="TestDataset", description="A dataset for testing"):
        if data is None:
            data = pd.DataFrame(
                {
                    "objects_per_image": [3,2,3,3,2,3,3,1,3],
                    "filepath": ["a", "b", "a", "a", "b", "c", "c", "d", "c"],
                    "label": ["Dog","Dog","Cat","Dog","Cat","Bird","Dog","Cat","Bird"]
                }
            )
        super().__init__(data, name=name, description=description)
    
    def __len__(self):
        return self.data.shape[0]
    
    # use properties as quick replacement for descriptors
    @property
    def data(self):
        return self.base_data
    
    @property
    def stats(self):
        return pd.DataFrame({"num_datapoints": [self.base_data.shape[0]]})
    
    @property
    def stats_dataset(self):
        return pd.DataFrame({"num_datapoints": [self.base_data.shape[0]]})
    
    @property
    def stats_images(self):
        return self.data
    
    @property
    def stats_classes(self):
        return pd.DataFrame({"num_datapoints": [self.base_data.shape[0]]})
    
    def save(self, export_path):
        print("Saved data at " + export_path)
    
    @classmethod
    def create_new_from_mask(cls, instance, mask):
        new_df = instance.data[mask]
        return cls(new_df, instance.name+"_new", instance.description+"_new")
    
test_dataset = TestDataset()

In [ ]:
#export
class Gallery(Dashboard, ABC):
    def __init__(self, dataset, sort_cols=None, sort_desciptor=None, width=500, height=500):
        if sort_cols is None and sort_desciptor is not None or sort_cols is not None and sort_desciptor is None:
            raise ValueError("sort_cols and sort_descrptor need to be both none or both need to be set.")
        else:
            self.sort_cols = sort_cols
            self.sort_desciptor = sort_desciptor
        self.index_mapping = [i for i in range(len(dataset))]
        self.dataset = dataset
        super().__init__(width, height)
        
    def get_mapped_index(self, idx):
        return self.index_mapping[idx]
        
    @abstractmethod
    def load_image_by_index(self, index, width, height):
        # needs to be a bokeh figure of pn Row, Column etc.
        pass
        
    def update_sorting(self, event):
        self.index_mapping = getattr(self.dataset, self.sort_desciptor)[event.obj.value].argsort()
        
    def build_gui(self):
        if self.sort_cols is not None:
            self.sorter = pnw.Select(name="Sort by", options=self.sort_cols)
            self.sorter.param.watch(self.update_sorting, "value")
        
        self.btn_prev = pnw.Button(name="<", width=int(2*self.width/6))
        self.btn_next = pnw.Button(name=">", width=int(2*self.width/6))
        self.current = pnw.TextInput(value="1", width=int(self.width/6))
        self.image_count = pn.Row("/" + str(len(self.dataset)), width=int(self.width/6))
        if self.sort_cols is not None:
            self.gui_controlls = pn.Column(self.sorter, pn.Row(self.btn_prev, self.current, self.image_count, self.btn_next, align="center", height=50))
        else:
            self.gui_controlls = pn.Row(self.btn_prev, self.current, self.image_count, self.btn_next, align="center", height=50)
        self._image = pn.Row(self.load_image_by_index(int(self.current.value)-1), align="center")
        self.gui = pn.Column(self.gui_controlls, self.image)
        
        self.btn_prev.on_click(self._previous)
        self.btn_next.on_click(self._next)
        
    @property
    def image(self):
        return self._image
    
    @image.setter
    def image(self, image):
        self._image = image
        self.gui[1] = self._image
        
    def _next(self, _):
        index = int(self.current.value)
        if index == len(self.dataset):
            index = 1
        else:
            index += 1
        self.current.value = str(index)
        self.image = pn.Row(self.load_image_by_index(index-1), align="center")
        
    def _previous(self, _):
        index = int(self.current.value)
        if index == 1:
            index = len(self.dataset)
        else:
            index -= 1
        self.current.value = str(index)
        self.image = pn.Row(self.load_image_by_index(index-1), align="center")
    
    def show(self):
        return self.gui

Creates a gallery with the images of a dataset. The dataset is required to have the following `method`:

- `load_image_by_index`: Takes an index and returns a bokeh figure or panel widget

In [ ]:
class TestGallery(Gallery):
    def load_image_by_index(self, index: int):
        index = self.get_mapped_index(index)
        return pn.Row(str(self.dataset.data["objects_per_image"][index]) + " - " + str(self.dataset.data["label"][index]))

test_gallery = TestGallery(test_dataset, sort_cols=["label", "objects_per_image"], sort_desciptor="stats_images").show()
assert test_gallery[1][0][0].object == "3 - Dog"
test_gallery[0][1][0].clicks += 1
test_gallery[0][1][0].clicks += 1
assert test_gallery[1][0][0].object == "1 - Cat"
test_gallery[0][1][-1].clicks += 1
test_gallery[0][1][-1].clicks += 1
assert test_gallery[1][0][0].object == "3 - Dog"

In [ ]:
test_gallery

In [ ]:
#export
class DatasetOverview(Dashboard):
    DESCRIPTOR_DATA = "data"
    DESCRIPTOR_STATS = "stats"
    
    def __init__(self, dataset: GenericDataset, height: int = 500, width: int = 500):
        """Creates an overview of a dataset"""
        self.dataset = dataset
        super().__init__(height=height, width=width)
        
    def _generate_dataset_tab(self):
        overview_table = table_from_dataframe(getattr(self.dataset, self.DESCRIPTOR_DATA), width=self.width, height=self.height)
        return overview_table
        
    def _generate_datset_stats_tab(self):
        overview_table = table_from_dataframe(getattr(self.dataset, self.DESCRIPTOR_STATS), width=self.width, height=self.height)
        return overview_table
    
    def build_gui(self):
        dataset_tab = self._generate_dataset_tab()
        dataset_stats_tab = self._generate_datset_stats_tab()
        self.gui = pn.Tabs(("Dataset overview", dataset_tab), ("Dataset stats overview", dataset_stats_tab), align="start")
        
    def show(self):
        return self.gui

The dataset is required to have to following (default) `descriptors`:

- **data** [pd.Dataframe]: Each row should be a singel datapoint/annotation
- **stats** [pd.Datagrame]: One row that describes the dataset, were the columns represent different information

In [ ]:
test_dataset_overview = DatasetOverview(test_dataset, height=200)
test_dataset_overview.show()

In [ ]:
#export
class MultiDatasetOverview(Dashboard):
    DESCRIPTOR_DATA = "stats"
    
    def __init__(self, datasets: Union[List[GenericDataset], ObservableList], height=100, width=1000, with_del_button=False):
        self.datasets = datasets if isinstance(datasets, ObservableList) else ObservableList(datasets)
        self.datasets.register_callback(self.update_table)
        self.with_del_button = with_del_button
        super().__init__(width=width, height=height)
    
    def create_overview_df(self):
        return pd.concat([getattr(dataset, self.DESCRIPTOR_DATA) for dataset in self.datasets]) if len(self.datasets) > 0 else pd.DataFrame() 
    
    def build_gui(self):
        self.delete_button = pnw.Button(name="Delete", width=self.width, height=25)
        self.overview_table = table_from_dataframe(self.create_overview_df(), height=self.height-25)
        if self.with_del_button:
            self.delete_button.on_click(self.delete_entry)
            self.gui = pn.Column(self.delete_button, self.overview_table)
        else:
            self.gui = pn.Column(self.overview_table)
        
    def delete_entry(self, clicks):
        selection = self.overview_table.selection
        self.datasets.list = [dataset for index, dataset in enumerate(self.datasets) if index not in selection]
    
    def update_table(self, event):
        self.overview_table.value = self.create_overview_df()
        self.overview_table.height = height=self.height-25
    
    def show(self):
        return self.gui

The dataset is required to have to following `descriptors`:

- **stats** [pd.Datagrame]: One row that descripbes the dataset, were the columns represent different information

In [ ]:
test_multi_dataset_overview = MultiDatasetOverview([test_dataset, test_dataset], with_del_button=True)
test_multi_dataset_overview.show()

In [ ]:
#export
class DatasetComparison(Dashboard):
    DESCRIPTOR_DATA = "data"
    DESCRIPTOR_STATS = "stats"
    
    def _get_descriptor_for_all_datasets(self, descriptor_name):
        return [getattr(dataset, descriptor_name) for dataset in self.datasets]
    
    def __init__(self, datasets: List[GenericDataset], height: int = 500, width: int = 500):
        """Creates an overview of a dataset"""
        self.datasets = datasets
        super().__init__(height=height, width=width)
        
    def _generate_dataset_tab(self):
        overview_table = table_from_dataframe(self._get_descriptor_for_all_datasets(self.DESCRIPTOR_DATA), width=self.width, height=self.height)
        return pn.Column(*overview_table)
        
    def _generate_datset_stats_tab(self):
        overview_table = table_from_dataframe(self._get_descriptor_for_all_datasets(self.DESCRIPTOR_STATS), width=self.width, height=self.height)
        return pn.Column(*overview_table)
    
    def build_gui(self):
        dataset_tab = self._generate_dataset_tab()
        dataset_stats_tab = self._generate_datset_stats_tab()
        self.gui = pn.Tabs(("Dataset overview", dataset_tab), ("Dataset stats overview", dataset_stats_tab), align="start")
        
    def show(self):
        return self.gui

The dataset is required to have to following (default) `descriptors`:

- **data** [pd.Dataframe]: Each row should be a singel datapoint/annotation
- **stats** [pd.Datagrame]: One row that describes the dataset, were the columns represent different information

In [ ]:
test_dataset_overview = DatasetComparison([test_dataset, test_dataset], height=200)
test_dataset_overview.show()

In [ ]:
#export
class DatasetFilter(Dashboard, ABC):
    DESCRIPTOR_DATA = "data"
    
    def __init__(self, dataset: GenericDataset, columns: Optional[List[str]] = None, height: int = 500, width: int = 500, filter_width: Optional[int] = None, filter_height: Optional[int] = None, n_cols: int = None):
        self.dataset = dataset
        self.columns = columns if columns is not None else getattr(self.dataset, self.DESCRIPTOR_DATA).columns
        self.n_cols = n_cols if n_cols is not None else ceil(len(self.columns)**0.5)
        self.n_rows = (len(self.columns)//self.n_cols) + min(1, len(self.columns)%self.n_cols)
        self.filter_height = int(height/self.n_rows) if filter_height is None else filter_height
        self.filter_width = int(width/self.n_cols) if filter_width is None else filter_width
        self.filters = []
        self.UPDATING = False
        super().__init__(height=height, width=width)
    
    def build_gui(self):
        """All filters used below need to have two functions get_selection and update_with_mask."""
        data_selection = getattr(self.dataset, self.DESCRIPTOR_DATA)[self.columns]
        self.generate_filters(data_selection)
        # put the images in the grid
        self.gui = pn.GridSpec(ncols=self.n_cols, nrows=self.n_rows, width=self.width, height=self.height)
        for index, gui_filter in enumerate(self.filters):
            self.gui[index//self.n_cols, index%self.n_cols] = gui_filter.show()
        # hook all control elements to the update functions
        for single_filter in self.filters:
            single_filter.register_callback(self.update_plots)
        
    @abstractmethod
    def generate_filters(self, dataselection):
        """Write handler for the different column types of the datagrame."""
        pass
        
    def _update_plots(self, current_selection):
        mask = np.array(self.get_selection())
        final_mask = np.logical_or(mask, current_selection)
        for single_filter in self.filters:
            single_filter.update_with_mask(final_mask)
        
    def update_plots(self, event, old=None, new=None):
        if self.UPDATING:
            return
        else:
            self.UPDATING = True
            self._update_plots(event)
            self.UPDATING = False
        
    def show(self):
        return self.gui
    
    def get_selection(self):
        mask = self.filters[0].get_selection()
        for single_filter in self.filters[1:]:
            mask = np.logical_and(mask, single_filter.get_selection())
        return mask
    
    def register_callback(self, callback):
        """Register callback to every underlying filter"""
        for filter in self.filters:
            filter.register_callback(callback)

The dataset is required to have to following (default) `descriptors`:

- **data** [pd.Dataframe]: Each row should be a singel datapoint/annotation with the columns pepresenting attributes to filter

In [ ]:
#export
class DatasetFilterWithRangeSliderAndMultiSelect(DatasetFilter):
    def generate_filters(self, data_selection):
        # generate filters
        for column in data_selection.columns:
            if pd.api.types.is_numeric_dtype(data_selection[column]):
                self.filters.append(RangeFilter(data_selection[column].values, column, height=self.filter_height, width=self.filter_width))
            elif pd.api.types.is_categorical_dtype(data_selection[column]) or pd.api.types.is_string_dtype(data_selection[column]):
                self.filters.append(CategoricalFilter(data_selection[column].values, column, height=self.filter_height, width=self.filter_width))

In [ ]:
test_dataset_filter = DatasetFilterWithRangeSliderAndMultiSelect(test_dataset, height=350)
test_dataset_filter.show()

In [ ]:
#export
class ScatterDatasetFilter(DatasetFilter):
    def __init__(self, dataset: GenericDataset, columns: Optional[List[str]] = None, height: int = 500, width: int = 500, filter_width: Optional[int] = None, filter_height: Optional[int] = None, n_cols: int = None):
        super().__init__(dataset, columns, height, width, filter_width, filter_height, n_cols)
        self.scatter_filter = None
    
    def build_gui(self):
        """All filters used below need to have two functions get_selection and update_with_mask."""
        data_selection = getattr(self.dataset, self.DESCRIPTOR_DATA)[self.columns]
        self.generate_filters(data_selection)
        # put the images in the grid
        self.categorical_grid = pn.GridSpec(ncols=self.n_cols, nrows=self.n_rows, width=self.width, height=self.height//2)
        for index, gui_filter in enumerate(self.filters):
            self.categorical_grid[index//self.n_cols, index%self.n_cols] = gui_filter.show()
        # hook all control elements to the update functions
        for single_filter in self.filters:
            single_filter.register_callback(self.update_plots)
        # add the scatter filter
        self.scatter_filter.register_callback(self.update_plots)
        self.gui = pn.Column(self.categorical_grid, self.scatter_filter.show())
    
    def generate_filters(self, data_selection):
        # generate filters
        numeric_cols = []
        for column in data_selection.columns:
            if pd.api.types.is_numeric_dtype(data_selection[column]):
                numeric_cols.append(column)
            elif pd.api.types.is_categorical_dtype(data_selection[column]) or pd.api.types.is_string_dtype(data_selection[column]):
                self.filters.append(CategoricalFilter(data_selection[column].values, column, height=self.filter_height, width=self.filter_width))
        # generate generic scatter selector
        self.scatter_filter = GenericMulitScatterFilter(data_selection[numeric_cols], height=self.height//2)

In [ ]:
test_dataset_filter = ScatterDatasetFilter(test_dataset, height=700)
test_dataset_filter.show()

In [ ]:
#export
class DatasetGenerator(Dashboard):
    DESCRIPTOR_DATA = "data"
    DESCRIPTOR_STATS = "stats"
    
    DATASET_FILTER_COLUMNS = None
    DATASET_FILTER = DatasetFilterWithRangeSliderAndMultiSelect
    MULTI_DATASET_OVERVIEW = MultiDatasetOverview
    DATASET_OVERVIEW = DatasetOverview

    def __init__(self, dataset, width=500, height=500):
        self.base_dataset = dataset
        self.created_datasets = ObservableList([])
        super().__init__(width, height)
        
    def build_gui(self):
        self.dataset_filter = self.DATASET_FILTER(self.base_dataset, columns=self.DATASET_FILTER_COLUMNS, width=self.width, height=self.height-50)
        self.dataset_filter_create_dataset_button = pnw.Button(name="Create", height=50)
        self.dataset_filter_create_dataset_button.on_click(self.create_dataset)
        self.dataset_filter_with_export = pn.Column(pn.Row(self.dataset_filter.show(), align="center"), pn.Row(self.dataset_filter_create_dataset_button, align="center"))
        
        self.created_datasets_overview = self.MULTI_DATASET_OVERVIEW(self.created_datasets, with_del_button=True, height=150, width=self.width)
        self.selected_dataset_overview = self.DATASET_OVERVIEW(self.base_dataset, height=self.height-250, width=self.width)
        self.export_gui = self.create_export_gui()
        self.datasets_overview = pn.Column(pn.Row(self.created_datasets_overview.gui, align="center"), pn.Row(self.selected_dataset_overview.gui, align="center"), pn.Row(self.export_gui, align="center"))
        self.created_datasets_overview.overview_table.param.watch(self.update_dataset_overview, "selection")        
        
        self.gui = pn.Tabs(("Dataset Filter", self.dataset_filter_with_export), ("Dataset Overview", self.datasets_overview))
        
    def create_export_gui(self):
        self.export_path = pnw.TextInput(name="Export path", value="datasets", height=50)
        self.export_button = pnw.Button(name="Export", align="end", height=50)
        self.export_button.on_click(self.export_datasets)
        
        export_dataset_name = "" if len(self.created_datasets) == 0 else self.created_datasets[self.export_dataset_overview.selection[0]].name
        export_description_name = "" if len(self.created_datasets) == 0 else self.created_datasets[self.export_dataset_overview.selection[0]].description
        self.export_name_input = pnw.TextInput(name="Dataset name", value=export_dataset_name, height=50)
        self.export_name_input.param.watch(self.change_dataset_name, "value")
        self.export_description_input = pnw.TextAreaInput(name="Description", value=export_description_name, height=50)
        self.export_description_input.param.watch(self.change_dataset_description, "value")
        return pn.Column(pn.Row(self.export_name_input, self.export_description_input), pn.Row(self.export_path, self.export_button))
        
    def change_dataset_name(self, event):
        index = self.created_datasets_overview.overview_table.selection[0]
        self.created_datasets[index].name = self.export_name_input.value
    
    def change_dataset_description(self, event):
        index = self.created_datasets_overview.overview_table.selection[0]
        self.created_datasets[index].description = self.export_description_input.value
        
    def update_dataset_overview(self, event):
        self.selected_dataset_overview = self.DATASET_OVERVIEW(self.created_datasets[event.new[0]], height=self.height-350)
        self.datasets_overview[1] = self.selected_dataset_overview.show()
        self.export_name_input.value = self.created_datasets[event.new[0]].name
        self.export_description_input.value = self.created_datasets[event.new[0]].description
        
    def create_dataset(self, clicks):
        mask = self.dataset_filter.get_selection()
        new_sub_dataset = self.base_dataset.create_new_from_mask(self.base_dataset, mask)
        self.created_datasets.append(new_sub_dataset)

    def export_datasets(self, clicks):
        export_path = self.export_path.value
        if not os.path.isdir(export_path):
            os.makedirs(export_path)
        for dataset in self.created_datasets:
            dataset.save(export_path)
        
    def show(self):
        return self.gui

The dataset is required to have to following (default) `descriptors`:

- **data** [pd.Datagrame]: Each row represents a datapoint
- **stats** [pd.Datagrame]: One row that descripbes the dataset, were the columns represent different information

Furthermore, the dataset set if required to have the following `functions`:

- **create_new_from_mask(class_instance, mask)**: Class function that is called with a mask and an instance of the class, that returns a new instance
- **save(save_path)**: Can be called with a path and save the dataset to that path

In [ ]:
dataset_generator = DatasetGenerator(test_dataset, height=700, width=500)
dataset_generator.show()